In [1]:
def deco(func):
  def inner():
    print('running inner()')
  return inner

@deco
def target():
  print("running target()")

target()

running inner()


In [2]:
target

<function __main__.deco.<locals>.inner()>

In [4]:
def logger(func):
  def wrapper(*args, **kwargs):
    print(f"[LOG] {func.__name__} called with args={args}, kwargs={kwargs}")
    result = func(*args, **kwargs)
    print(f"[LOG] {func.__name__} returned {result}")
    return result
  return wrapper

@logger
def add(a, b):
  return a + b

add(5, 12)

[LOG] add called with args=(5, 12), kwargs={}
[LOG] add returned 17


17

In [6]:
registry = []

def register(func):
  print('running register(%s)' % func)
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

def f3():
  print('running f3()')

def main():
  print('running main()')
  print('registry ->', registry)
  f1()
  f2()
  f3()

if __name__=='__main__':
  main()

running register(<function f1 at 0x7aed351c58a0>)
running register(<function f2 at 0x7aed351c6840>)
running main()
registry -> [<function f1 at 0x7aed351c58a0>, <function f2 at 0x7aed351c6840>]
running f1()
running f2()
running f3()


In [7]:
promos = []

def promotion(promo_func):
  promos.append(promo_func)
  return promo_func

@promotion
def fidelity_promo(order):
  return order.total() * 0.05 if order.custome.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

@promotion
def large_order_promo(order):
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

def best_promo(order):
  return max(promo(order) for promo in promos)


In [8]:
def f1(a):
  print(a)
  print(b)

f1(3)

3


NameError: name 'b' is not defined

In [9]:
b=6
f1(3)

3
6


In [10]:
b = 6
def f2(a):
  print(a)
  print(b)
  b = 9

f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [11]:
b = 6
def f3(a):
  global b
  print(a)
  print(b)
  b = 9

f3(3)

3
6


In [12]:
b

9

In [13]:
f3(3)

3
9


In [14]:
b = 30
f3(3)

3
30


In [15]:
class Averager():
  def __init__(self):
    self.series = []

  def __call__(self, new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total/len(self.series)

In [16]:
avg = Averager()
avg(10)

10.0

In [17]:
avg(11)

10.5

In [18]:
avg(12)

11.0

In [19]:
def make_averager():
  series = []

  def averager(new_value):
    series.append(new_value)
    total = sum(series)
    return total/len(series)

  return averager

In [20]:
avg = make_averager()
avg(10)

10.0

In [21]:
avg(11)

10.5

In [22]:
avg(12)

11.0

In [23]:
avg.__code__.co_varnames

('new_value', 'total')

In [24]:
avg.__code__.co_freevars

('series',)

In [25]:
avg.__closure__

(<cell at 0x7aed379bc670: list object at 0x7aed33ebabc0>,)

In [26]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [27]:
def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    count += 1
    total += new_value
    return total / count

  return averager

In [28]:
avg = make_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [29]:
def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total / count

  return averager

In [30]:
avg = make_averager()
avg(10)

10.0

In [31]:
import time

def clock(func):
  def clocked(*args):
    t0 = time.perf_counter()
    result = func(*args)
    elapsed = time.perf_counter() - t0
    name = func.__name__
    arg_str = ', '.join(repr(arg) for arg in args)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [32]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
  print('*'*40, 'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12310789s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000085s] factorial(1) -> 1
[0.00001929s] factorial(2) -> 2
[0.00002940s] factorial(3) -> 6
[0.00003959s] factorial(4) -> 24
[0.00005002s] factorial(5) -> 120
[0.00005994s] factorial(6) -> 720
6! = 720


In [33]:
factorial.__name__

'clocked'

In [34]:
import time
import functools

def clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time() - t0
    arg_lst = []
    name = func.__name__
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k,w) for k,w in sorted(kwargs.items())]
      arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(arg_lst)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [36]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
  print('*'*40, 'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12315536s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000048s] factorial(1) -> 1
[0.00001240s] factorial(2) -> 2
[0.00001979s] factorial(3) -> 6
[0.00002742s] factorial(4) -> 24
[0.00003576s] factorial(5) -> 120
[0.00004578s] factorial(6) -> 720
6! = 720


In [37]:
factorial.__name__

'factorial'